# Data Mining Project

### Imports

In [162]:
from numpy import genfromtxt
import pandas as pd
import numpy as np
import operator, re, string, codecs, nltk
from statistics import mean
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('punkt')
from xml.dom import minidom
from string import punctuation
from enum import Enum
try:
    maketrans = ''.maketrans
except AttributeError:
    from string import maketrans
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from collections import Counter

# Test

# from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn import decomposition, ensemble

# import xgboost, textblob
# from keras.preprocessing import text, sequence
# from keras import layers, models, optimizers

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Program base

In [273]:

class Type(Enum):
    REAL_CSV_SETS = 1
    TEST_ON_TRAINING_SET = 2

class ArticlePredictorBase:
    def __init__(self,test_type,path_to_csv_train,path_to_csv_test):
        if Type.REAL_CSV_SETS == test_type:
            
            #Import of the test set
            try:
                test_csv_file = path_to_csv_test
                test = pd.read_csv(test_csv_file,delimiter=',')
                
                X = test.iloc[:,1]
                y = test.iloc[:,2]
                print(X)
                print(y)
            except:
                print("Error while importing testing set csv file") 
                self.type = None
            
            #Import of the train set
            try:
                train_csv_file = path_to_csv_train
                train = pd.read_csv(train_csv_file,delimiter=',')
                #TODO self.train_x, self.train_y in this case
                print("Not yet implemented")
            except:
                print("Error while importing training set csv file") 
                self.type = None
                
            self.type = test_type
                
        elif Type.TEST_ON_TRAINING_SET == test_type:
            
            #Import of the train/testing set
            try:
                train_csv_file = path_to_csv_train
                data = pd.read_csv(train_csv_file,delimiter=',')
            except:
                print("Error while importing the csv file") 
                self.type = None
                
            self.type = test_type
                
            X = data.iloc[:,1]
            y = data.iloc[:,2]
            
            self.train_x, self.test_x, self.train_y, self.test_y = model_selection.train_test_split(X, y)
        else:
            self.type = None
            print('Error type')

    def preprocess(self, set_x, set_y):
        the_set = pandas.DataFrame()
        the_set['file_name'] = set_x
        the_set['label'] = set_y
        
        for file in the_set['file_name']:
            print(file)
            file_path = 'project/project/data/'+file['file_name']
            #excepted = train['label'][i]
            print(file_path)
            #xmldoc = minidom.parse(file_path)
            
            #TODO preprocess (accès au fichier, compute des trucs, ajouter des champs dans le set)
            
        return set_x, set_y
            
    def test(self):
        print("(Testing Launched)")
        if self.type is None:
            print("Initialisation error ")
            return None

        test_x, test_y = self.preprocess(self.test_x, self.test_y)
        #TODO testing
        
    def train(self):
        print("(Training Launched)")
        if self.type is None:
            print("Initialisation error ")
            return None
            
        train_x, train_y = self.preprocess(self.train_x, self.train_y)
        #TODO training
            
            
predictor = ArticlePredictorBase(Type.REAL_CSV_SETS,"project/project/train.csv","project/project/test.csv")
predictor = ArticlePredictorBase(Type.TEST_ON_TRAINING_SET,"project/project/train.csv",None)
#predictor.train()

0        20.xml
1        47.xml
2        57.xml
3        66.xml
4        88.xml
5        97.xml
6       127.xml
7       165.xml
8       193.xml
9       199.xml
10      215.xml
11      240.xml
12      303.xml
13      397.xml
14      410.xml
15      463.xml
16      484.xml
17      553.xml
18      573.xml
19      574.xml
20      599.xml
21      622.xml
22      640.xml
23      689.xml
24      697.xml
25      740.xml
26      746.xml
27      832.xml
28      842.xml
29      884.xml
         ...   
170    4107.xml
171    4153.xml
172    4156.xml
173    4169.xml
174    4187.xml
175    4234.xml
176    4266.xml
177    4272.xml
178    4292.xml
179    4347.xml
180    4355.xml
181    4359.xml
182    4365.xml
183    4377.xml
184    4381.xml
185    4513.xml
186    4545.xml
187    4565.xml
188    4583.xml
189    4609.xml
190    4634.xml
191    4655.xml
192    4671.xml
193    4674.xml
194    4704.xml
195    4741.xml
196    4819.xml
197    4872.xml
198    4944.xml
199    4960.xml
Name: file, Length: 200,

## Computation of 1 file

We test here with 1.xml data file

### Importation of the file

In [165]:
xmldoc = minidom.parse('project/project/data/1.xml')
itemlist = xmldoc.getElementsByTagName('BODY')

#Verification

if len(itemlist) != 1:
    print('Error: XML file invalid')
    sys.exit(0)

text = itemlist[0].childNodes[0].nodeValue

### Preprocessing

In [166]:
#Preprocess
text = text.lower()    
text = re.sub(r'\d+', '', text) #Deleting numbers
text = text.translate(str.maketrans('','',string.punctuation)) #Deleting ponctuation

#Preprocess ideas : https://medium.com/@datamonsters/text-preprocessing-in-python-steps-tools-and-examples-bf025f872908
     
#Tokenization    
tokens = word_tokenize(text)
print(str(len(tokens)) + " tokens BEFORE stop words removing")
tokens = [i for i in tokens if not i in ENGLISH_STOP_WORDS]
print(str(len(tokens)) + " tokens AFTER stop words removing")

#print(tokens)

#stemmer= PorterStemmer()
#tokens = set(map(stemmer.stem,tokens))
#print (tokens)

438 tokens BEFORE stop words removing
248 tokens AFTER stop words removing


In [167]:
def most_common_terms(terms):
    terms_count_array = []
    for term in terms:
        terms_count_array += term.split(" ")
    counter = Counter(terms_count_array)
    for word in counter.most_common():
        print(word)
        
most_common_terms(tokens)

('dlrs', 14)
('new', 9)
('york', 8)
('sales', 7)
('times', 7)
('cocoa', 6)
('comissaria', 5)
('smith', 5)
('said', 5)
('bags', 5)
('mln', 5)
('crop', 5)
('bahia', 4)
('february', 3)
('total', 3)
('ports', 3)
('junejuly', 3)
('augsept', 3)
('marchapril', 3)
('octdec', 3)
('dec', 3)
('week', 2)
('temporao', 2)
('period', 2)
('year', 2)
('arrivals', 2)
('kilos', 2)
('consignment', 2)
('figures', 2)
('farmers', 2)
('shippers', 2)
('sold', 2)
('bean', 2)
('shipment', 2)
('limited', 2)
('tonne', 2)
('open', 2)
('july', 2)
('butter', 2)
('sept', 2)
('currency', 2)
('areas', 2)
('uruguay', 2)
('showers', 1)
('continued', 1)
('zone', 1)
('alleviating', 1)
('drought', 1)
('early', 1)
('january', 1)
('improving', 1)
('prospects', 1)
('coming', 1)
('normal', 1)
('humidity', 1)
('levels', 1)
('restored', 1)
('weekly', 1)
('review', 1)
('dry', 1)
('means', 1)
('late', 1)
('ended', 1)
('making', 1)
('cumulative', 1)
('season', 1)
('stage', 1)
('delivered', 1)
('earlier', 1)
('included', 1)
('doubt', 